In [16]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pylab
import numpy as np
from qiskit import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, HFMethodType
from qiskit.chemistry import FermionicOperator
from groupedFermionicOperator import groupedFermionicOperator
from qiskit.aqua.operators import WeightedPauliOperator


In [2]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
dist =1.4
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.BOHR, 
                         charge=0, spin=0, basis='sto-3g')
molecule = driver.run()
num_electrons = molecule.num_alpha + molecule.num_beta

In [3]:
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
ferOp.h1

array([[-1.25279706,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.4756023 ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.25279706,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.4756023 ]])

In [4]:
ferOp.h2

array([[[[-0.33729704,  0.        ,  0.        ,  0.        ],
         [ 0.        , -0.09062896,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[ 0.        , -0.09062896,  0.        ,  0.        ],
         [-0.331782  ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [-0.33729704,  0.        ,  0.        ,  0.        ],
         [ 0.        , -0.09062896,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        , -0.09062896,  0.        ,  0.        ],
         [-0.331782  ,  0.        ,  0.        , 

In [5]:
molecule.num_orbitals*2

4

In [6]:
from groupedFermionicOperator import groupedFermionicOperator

g = groupedFermionicOperator(ferOp, num_electrons,mode="uhf")
qubitOp = g.to_paulis()

group.op
{(0, 0): -1.6807230613940565, (1, 1): -0.9149789301108968, (2, 2): -1.6807230613940565, (3, 3): -0.9149789301108968, ((0, 0), (0, 0)): 0.33729704216168466, ((0, 1), (1, 0)): 0.09062895739655422, ((0, 1), (0, 1)): 0.09062895739655424, ((0, 0), (1, 1)): 0.33178199561027416, ((0, 0), (2, 2)): 0.33729704216168466, ((0, 1), (3, 2)): 0.09062895739655422, ((0, 1), (2, 3)): 0.09062895739655424, ((0, 0), (3, 3)): 0.33178199561027416, ((1, 1), (0, 0)): 0.3317819956102742, ((1, 0), (1, 0)): 0.0906289573965542, ((1, 0), (0, 1)): 0.09062895739655424, ((1, 1), (1, 1)): 0.348747673340091, ((1, 1), (2, 2)): 0.3317819956102742, ((1, 0), (3, 2)): 0.0906289573965542, ((1, 0), (2, 3)): 0.09062895739655424, ((1, 1), (3, 3)): 0.348747673340091, ((2, 2), (0, 0)): 0.33729704216168466, ((2, 3), (1, 0)): 0.09062895739655422, ((2, 3), (0, 1)): 0.09062895739655424, ((2, 2), (1, 1)): 0.33178199561027416, ((2, 2), (2, 2)): 0.33729704216168466, ((2, 3), (3, 2)): 0.09062895739655422, ((2, 3), (2, 3)): 0.0906

0.348747673340091
((3, 3), (3, 3)) [[(0.375+0j), Pauli('III')], [(-0.125+0j), Pauli('IIZ')], [(0.125+0j), Pauli('IZI')], [(0.125+0j), Pauli('IZZ')], [(-0.125+0j), Pauli('ZII')], [(-0.125+0j), Pauli('ZIZ')], [(-0.375+0j), Pauli('ZZI')], [(0.125+0j), Pauli('ZZZ')]]



In [17]:
qubitOp

In [18]:
qubitOp.to_dict()

{'paulis': [{'label': 'III',
   'coeff': {'real': -0.8383208251201104, 'imag': 0.0}},
  {'label': 'IIZ', 'coeff': {'real': 0.19716134840999316, 'imag': 0.0}},
  {'label': 'IZI', 'coeff': {'real': -0.3059031401253504, 'imag': 0.0}},
  {'label': 'IZZ', 'coeff': {'real': 0.19716134840999314, 'imag': 0.0}},
  {'label': 'ZII', 'coeff': {'real': -0.46337019090737325, 'imag': 0.0}},
  {'label': 'ZIZ', 'coeff': {'real': -0.05093465976858392, 'imag': 0.0}},
  {'label': 'ZZI', 'coeff': {'real': 0.06904749408738717, 'imag': 0.0}},
  {'label': 'ZZZ', 'coeff': {'real': -0.05093465976858394, 'imag': 0.0}},
  {'label': 'IIX', 'coeff': {'real': 0.04531447869827711, 'imag': 0.0}},
  {'label': 'IZX', 'coeff': {'real': -0.04531447869827711, 'imag': 0.0}},
  {'label': 'ZIX', 'coeff': {'real': 0.04531447869827711, 'imag': 0.0}},
  {'label': 'ZZX', 'coeff': {'real': -0.04531447869827711, 'imag': 0.0}},
  {'label': 'XIX', 'coeff': {'real': 0.04531447869827711, 'imag': 0.0}},
  {'label': 'XZX', 'coeff': {'rea

In [19]:
dict = {'paulis': [{"coeff": {"imag": 0.00, "real": 999 }, "label": "III" }]}
b = WeightedPauliOperator.from_dict(dict)

In [20]:
qubitOp.add(b)

In [22]:
qubitOp.to_dict()

{'paulis': [{'label': 'III',
   'coeff': {'real': 998.1616791748799, 'imag': 0.0}},
  {'label': 'IIZ', 'coeff': {'real': 0.19716134840999316, 'imag': 0.0}},
  {'label': 'IZI', 'coeff': {'real': -0.3059031401253504, 'imag': 0.0}},
  {'label': 'IZZ', 'coeff': {'real': 0.19716134840999314, 'imag': 0.0}},
  {'label': 'ZII', 'coeff': {'real': -0.46337019090737325, 'imag': 0.0}},
  {'label': 'ZIZ', 'coeff': {'real': -0.05093465976858392, 'imag': 0.0}},
  {'label': 'ZZI', 'coeff': {'real': 0.06904749408738717, 'imag': 0.0}},
  {'label': 'ZZZ', 'coeff': {'real': -0.05093465976858394, 'imag': 0.0}},
  {'label': 'IIX', 'coeff': {'real': 0.04531447869827711, 'imag': 0.0}},
  {'label': 'IZX', 'coeff': {'real': -0.04531447869827711, 'imag': 0.0}},
  {'label': 'ZIX', 'coeff': {'real': 0.04531447869827711, 'imag': 0.0}},
  {'label': 'ZZX', 'coeff': {'real': -0.04531447869827711, 'imag': 0.0}},
  {'label': 'XIX', 'coeff': {'real': 0.04531447869827711, 'imag': 0.0}},
  {'label': 'XZX', 'coeff': {'real'

In [8]:
repulsion_energy = molecule.nuclear_repulsion_energy
repulsion_energy

0.7142857142857143

In [9]:
from qiskit.aqua.algorithms import NumPyEigensolver
result_exact = NumPyEigensolver(qubitOp).run()
energy_exact = (np.real(result_exact.eigenvalues))[0]
print(energy_exact+repulsion_energy)

qubitOpJW = ferOp.mapping(map_type='jordan_wigner', threshold=1e-6)
result_exactJW = NumPyEigensolver(qubitOpJW).run()
energy_exactJW = (np.real(result_exactJW.eigenvalues))[0]
print(energy_exactJW+repulsion_energy)

-1.1372759436170456
-1.1372759436170452
